# Creating a web app to deploy the model

In [1]:
!pip install wandb
!pip install onnxruntime
!pip install -q gradio 

     |████████████████████████████████| 1.7 MB 7.1 MB/s 
     |████████████████████████████████| 139 kB 63.6 MB/s 
     |████████████████████████████████| 180 kB 40.7 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=e3ba50733f4c56f88c90419a491959e680e375cbf6537be0e7f121aa0d7ae425
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=3712aa3f71529dea784e86c083e7d7a33f2f925ca1e8702a51c7a5c7f874c5df
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 4.8 MB 8.8 MB/s 
     |████████████████████████████████| 3.6 MB 7.0 MB/s 
     |███████████████████████

In [2]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr

## Initialize Weights and Biases

In [3]:
os.environ["WANDB_API_KEY"] = "8e54e1439d9cdf18a32eab0f9d46a9d83fd20f69"

In [4]:
run = wandb.init(project='usedcar')

wandb: Currently logged in as: manaranjan (use `wandb login --relogin` to force relogin)


## Download the model and load it

In [5]:
ARTIFACT_NAME = 'XGBoost_UserCar:v0'

In [6]:
#run = wandb.init()
artifact = run.use_artifact('manaranjan/usedcar/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()

In [7]:
artifact_dir

'./artifacts/XGBoost_UserCar:v0'

In [8]:
!ls -al  artifacts/XGBoost_UserCar:v0

total 300
drwxr-xr-x 2 root root   4096 Oct 22 08:50 .
drwxr-xr-x 3 root root   4096 Oct 22 08:50 ..
-rw-r--r-- 1 root root 296292 Oct 22 08:50 usedcar_xgboost.onnx


http://onnx.ai/sklearn-onnx/auto_examples/plot_complex_pipeline.html#example-complex-pipeline

## Columns

In [9]:
x_columns = ['KM_Driven', 'Fuel_Type', 'age',
              'Transmission', 'Owner_Type', 'Seats',
              'make', 'mileage_new', 'engine_new', 'model',
              'power_new', 'Location']

cat_features = ['Fuel_Type',
                'Transmission', 'Owner_Type', 'model',
                'make', 'Location']

num_features = list(set(x_columns) - set(cat_features))                              

## Implement the predict() function

The followng inputs need to be supplied:

['KM_Driven', 'Fuel_Type', 'age',
'Transmission', 'Owner_Type', 'Seats',
'make', 'mileage_new', 'engine_new', 'model',
'power_new', 'Location']

In [10]:
def predict_price(make, model, fuelType,
                  transmission, engine, power, 
                  seats, mileage, kmDriven, 
                  age, ownerType, location):

    inputs_dict = {'KM_Driven' : float(kmDriven), 
              'Fuel_Type': fuelType, 
              'age': float(age), 
              'Transmission': transmission, 
              'Owner_Type': ownerType, 
              'Seats': float(seats), 
              'make': make.lower(), 
              'mileage_new': float(mileage), 
              'engine_new': float(engine), 
              'model': model.lower(), 
              'power_new': float(power), 
              'Location': location}

    df = pd.DataFrame(inputs_dict, index = [0])
    print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))            
  
    sess = rt.InferenceSession(artifact_dir + '/usedcar_xgboost.onnx')
    pred_onx = sess.run(None, inputs)

    predicted_price = float(pred_onx[0][0,0])
    return {f'Expected sale price of the car is: INR {np.round(predicted_price, 2)} lakhs' }

## Implement UI

In [11]:
kmDriven = gr.inputs.Slider(minimum=10, maximum=200, label="KM Driven (in 1000s kms)")
mileage = gr.inputs.Slider(minimum=10, maximum=40, label="Mileage")
engine = gr.inputs.Slider(minimum=600, maximum=1000, label="Engine Capacity (in cc)")
power = gr.inputs.Slider(minimum=40, maximum=100, label="Power (in bhp)")
age = gr.inputs.Dropdown(list(range(1, 10)), default="1", label="Age of the car ((in years): ")


fuelType = gr.inputs.Radio(['Petrol', 'Diesel'], default="Petrol", label="Fuel Type")
transmission = gr.inputs.Radio(['Manual', 'Automatic'], default="Manual", label="Transmission")
ownerType = gr.inputs.Radio(['First', 'Second'], default="First", label="Owner Type")

seats = gr.inputs.Dropdown(list(range(1, 7)), default="4", label="Number of Seats")

make = gr.inputs.Radio(['Maruti'], default="Maruti", label="Make")
model = gr.inputs.Radio(['Alto', 'Swift', 'Desire', 'Zen'], default="Swift", label="Model")
location = gr.inputs.Radio(['Bangalore', 'Hyderabad', 'Mumbai', 'Chennai'], default="Bangalore", label="Make")

fare = gr.inputs.Slider(minimum=0, maximum=1000, default=100, label="Fare (british pounds)")

gr.Interface(predict_price, [make, model, fuelType, transmission, engine, 
                             power, seats, mileage, kmDriven, age,
                             ownerType, location], "text", live=False).launch(debug=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://53835.gradio.app


   KM_Driven Fuel_Type  age  ...  model power_new   Location
0       57.0    Petrol  4.0  ...  swift      85.0  Bangalore

[1 rows x 12 columns]


KeyboardInterrupt: ignored

## PMML Approach
- An alternative approach of using PMML can be demoed. 
  - https://github.com/SoftwareAG/nyoka

In [ ]:
a = np.array([[2.0]]).astype(np.float32)

In [ ]:
a

array([[2.]], dtype=float32)

In [ ]:
a[0,0]

2.0